## *Web-Scrapp using rss links*

In [20]:
# Importing Libraries
import pandas as pd
import numpy as np
import xml.etree.cElementTree as et

In [21]:
# Parsing
tree=et.parse('terkini.xml')
root=tree.getroot()

In [22]:
# Empty list of each data that will be filled
Title = []
Link = []
Timestamp = []
Detail_article = []

In [23]:
# Reading
for title in root.iter('title'):
    Title.append(title.text)
    
for url in root.iter('link'):
    Link.append(url.text)
    
for pubdate in root.iter('pubDate'):
    Timestamp.append(pubdate.text)
    
for detail in root.iter('description'):
    Detail_article.append(detail.text)

In [24]:
# Data in dataframe
News_detail_df = pd.DataFrame(list(zip(Title, Link, Timestamp, Detail_article)), 
                       columns=['Title', 'Link', 'Timestamp', 'Detail_article'])

In [25]:
# top 5 data scrapped
News_detail_df.head()

,Title,Link,Timestamp,Detail_article
0,ANTARA News Jambi Terkini,https://jambi.antaranews.com,"Sun, 28 Aug 2022 21:39:36 +0700",News And Service from ANTARA News Jambi
1,Malisya Deslilian lulus Fakultas Kedokteran Un...,https://jambi.antaranews.com/berita/519381/mal...,"Sun, 28 Aug 2022 21:22:25 +0700","<img src=""https://img.antaranews.com/cache/255..."
2,Polda Jambi perkuat pembinaan sikap personel u...,https://jambi.antaranews.com/berita/519377/pol...,"Sun, 28 Aug 2022 20:42:00 +0700","<img src=""https://img.antaranews.com/cache/255..."
3,Ribuan peserta antusias ikuti Jambi Color Run ...,https://jambi.antaranews.com/berita/519373/rib...,"Sun, 28 Aug 2022 20:27:58 +0700","<img src=""https://img.antaranews.com/cache/255..."
4,Pertamina Patra Niaga Regional Sumbagsel sesal...,https://jambi.antaranews.com/berita/519369/per...,"Sun, 28 Aug 2022 19:18:58 +0700","<img src=""https://img.antaranews.com/cache/255..."


## *Saving the news detail to a database*

In [2]:
# importing libraries
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [3]:
# Connecting to MySQL Server
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [7]:
# Connecting
connection = create_server_connection("localhost", "root", "insert your mysql password here")

MySQL Database connection successful


In [8]:
# Function for Creating a New Database
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

In [9]:
# Creating a New Database
create_database_query = "CREATE DATABASE news_article"
create_database(connection, create_database_query)

Database created successfully


Now that we have created a database in MySQL Server, we can modify our create_server_connection function to connect directly to this database. Note that it's possible - common, in fact - to have multiple databases on one MySQL Server, so we want to always and automatically connect to the database we're interested in.

We can do this like so:

In [10]:
# Connecting to the Database
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [11]:
# Creating a Query Execution Function
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [18]:
# Creating Tables
create_newsarticle_table = """
CREATE TABLE news_article (
  id int(11) NOT NULL AUTO_INCREMENT,
  title varchar(255) COLLATE utf8mb4_unicode_520_ci NOT NULL DEFAULT '',
  url varchar(255) CHARACTER SET latin1 NOT NULL DEFAULT '',
  content longtext COLLATE utf8mb4_unicode_520_ci,
  summary text COLLATE utf8mb4_unicode_520_ci,
  article_ts bigint(20) NOT NULL DEFAULT '0' COMMENT 'published timestamp of article',
  published_date date DEFAULT NULL,
  inserted timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,
  updated timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
  PRIMARY KEY (id),
  UNIQUE KEY UNIK (url)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_520_ci;
 """

connection = create_db_connection("localhost", "root", "insert your mysql password here", "news_article") # Connect to the Database
execute_query(connection, create_newsarticle_table) # Execute our defined query

MySQL Database connection successful
Query successful


*Since the task is `to import the news details to a database`, I will perform how to make a query to import the information `manually`.*

In [29]:
# Populating the Tables
pop_newsarticle = """
INSERT INTO news_article VALUES
(1, 'ANTARA News Jambi Terkini', 'https://jambi.antaranews.com', 'News And Service from ANTARA News Jambi', NULL, NULL, 'Sun, 28 Aug 2022 21:39:36 +0700', NULL, NULL);
"""

connection = create_db_connection("localhost", "root", "insert your mysql password here", "news_article")
execute_query(connection, pop_newsarticle)


MySQL Database connection successful
Error: '1048 (23000): Column 'article_ts' cannot be null'


***Column `article_ts`*** cannot be null, I inserted `NULL` values because I'm still searching for how to convert the string value (*since the timestamp scrapped result is in `string` form*), I'll fix it later soon.

But at least we've near to reach the goal that is to scrapp the news articles using rss links and importing the result on a database.